### Part 1

Use “midterm_partone.csv” file that contains the stock-return information of small retailers    
(the same as the one we used in Session 5). Suppose that an industry expert (e.g., David Berman)   
claims that there is a bias in moment conditions of instrumental variables such that   
𝑍!(𝑌 − 𝑋𝐵) = 𝛿 *   
111
,, where 𝛿 has a non-zero value.
1. Update the GMM model that we discussed in class by incorporating the 𝛿 term to the
instrumental-variable moment expressions.
2. By analyzing the GMM summary table and test statistics of coefficients, determine if the
industry expert’s claim is statistically justified.

In [6]:
import numpy as np
import pandas as pd
import matplotlib as mp
import statsmodels.api as sm

from statsmodels.sandbox.regression.gmm import IV2SLS 
# There is a package named IV2SLS in Python. Do not use this package! The exogenous explanatory variables must
# be entered as instruments. So it gives wrong answers
from statsmodels.sandbox.regression.gmm import GMM

In [7]:
df1 = pd.read_csv(r"D:\1.5110 Predictive Modeling\midterm_partone.csv")
df1.head()

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446


In [8]:
model_iv = sm.OLS(df1["Inventory Turnover"],df1[["Constant","Current Ratio","Quick Ratio",\
                                                                 "Debt Asset Ratio"]]).fit()
endog_predict = model_iv.predict(df1[["Constant","Current Ratio","Quick Ratio","Debt Asset Ratio"]])
df1["Endogenous Param"] = endog_predict

In [9]:
model_2sls = sm.OLS(df1["Stock Change"], df1[["Constant","Endogenous Param",\
                                                              "Operating Profit","Interaction Effect",\
                                                             ]]).fit()
model_2sls.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Stock Change   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     8.530
Date:                Wed, 08 Nov 2023   Prob (F-statistic):           1.27e-05
Time:                        20:49:54   Log-Likelihood:                -1186.5
No. Observations:                1696   AIC:                             2381.
Df Residuals:                    1692   BIC:                             2403.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Constant              -0.0176      0.020     -0.896      0.370      -0.056       0.021
Endogenous Param       0.0011      0.001      1.827      0.068   -7.76e-05       0.002
Operating Profit      -0.1201      0.028     -4.319      0.000      -0.175      -0.066
Interaction Effect     0.0014      0.000      3.621      0.000       0.001       0.002
==============================================================================
Omnibus:                      368.832   Durbin-Watson:                   2.243
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3433.920
Skew:                           0.742   Prob(JB):                         0.00
Kurtosis:                       9.811   Cond. No.                         109.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
y_vals  = np.array(df1["Stock Change"])
x_vals  = np.array(df1[["Inventory Turnover","Operating Profit","Interaction Effect"]])
iv_vals = np.array(df1[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] 
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] 
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] 

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g


beta0 = np.array([0.1, 0.1, 0.1, 0.1])
res = gmm(endog = y_vals, exog = x_vals, instrument = iv_vals, k_moms=6, k_params=4).fit(beta0)

res.summary()

Optimization terminated successfully.
         Current function value: 0.000046
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.000373
         Iterations: 7
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.6317
Model:                            gmm   Prob (Hansen J):                 0.729
Method:                           GMM                                         
Date:                Wed, 08 Nov 2023                                         
Time:                        20:49:54                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0200      0.021     -0.964      0.335      -0.061       0.021
p 1            0.0011      0.001      1.843      0.065   -6.89e-05       0.002
p 2           -0.1071      0.032     -3.370      0.001      -0.169      -0.045
p 3            0.0011      0.000      2.760      0.006       0.000       0.002
==============================================================================
"""

### Part 2

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [12]:
# Load the dataset
df2 = pd.read_csv(r"D:\1.5110 Predictive Modeling\midterm_parttwo.csv")

In [17]:
df2.head()

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative


In [18]:
# Split the dataset into training and test sets (50% each)
train_data, test_data = train_test_split(df2, test_size=0.5, random_state=42)

In [19]:
# Define the features and target variable
features = ["Requested Credit Amount", "Marital Status", "Number of Dependents", 
            "Years of Education after High School", "Monthly Income", "Monthly Expense"]
target = "Credit Rating"

In [20]:
# Encode categorical variables using one-hot encoding
train_data = pd.get_dummies(train_data, columns=features, drop_first=True)
test_data = pd.get_dummies(test_data, columns=features, drop_first=True)

In [21]:
# Fit a logistic regression model
model = LogisticRegression()
model.fit(train_data.drop(columns=[target]), train_data[target])

LogisticRegression()

In [22]:
# Predict credit ratings on the test set
predictions = model.predict(test_data.drop(columns=[target]))

In [23]:
# Calculate confusion matrix, recall, precision, and F1 score
conf_matrix = confusion_matrix(test_data[target], predictions)
precision = precision_score(test_data[target], predictions, pos_label='Positive')
recall = recall_score(test_data[target], predictions, pos_label='Positive')
f1 = f1_score(test_data[target], predictions, pos_label='Positive')

In [24]:
print("Confusion Matrix:")
print(conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Confusion Matrix:
[[   0  577]
 [   0 3464]]
Precision: 0.8572135609997525
Recall: 1.0
F1 Score: 0.9231179213857428


In [25]:
# Calculate the threshold for the top 15% of predictions
threshold = sorted(model.predict_proba(test_data.drop(columns=[target]))[:, 1], reverse=True)[int(len(test_data) * 0.15)]

In [26]:
# Update the predictions based on the new threshold
new_predictions = [1 if p >= threshold else 0 for p in model.predict_proba(test_data.drop(columns=[target]))[:, 1]]

In [29]:
# Calculate updated confusion matrix, recall, precision, and F1 score
new_conf_matrix = confusion_matrix(test_data[target], new_predictions, labels=["Negative", "Positive"])
new_precision = precision_score(test_data[target], new_predictions, labels=["Negative", "Positive"], pos_label="Positive")
new_recall = recall_score(test_data[target], new_predictions, labels=["Negative", "Positive"], pos_label="Positive")
new_f1 = f1_score(test_data[target], new_predictions, labels=["Negative", "Positive"], pos_label="Positive")

print("\nUpdated Confusion Matrix:")
print(new_conf_matrix)
print("Updated Precision:", new_precision)
print("Updated Recall:", new_recall)
print("Updated F1 Score:", new_f1)


TypeError: Labels in y_true and y_pred should be of the same type. Got y_true=['Negative' 'Positive'] and y_pred=[0 1]. Make sure that the predictions provided by the classifier coincides with the true labels.